# Using the Brier score to evaluate the predictive ability of a time-to-event model

References:
- [scikit-survival docs - Evaluating survival models](https://scikit-survival.readthedocs.io/en/stable/user_guide/evaluating-survival-models.html#Time-dependent-Brier-Score)
    - Discusses the bias of Harrell's c-index, and the fact that it is not a useful measure of performance if a specific time range is of primary interest (e.g. predicting death within 2 years) 
    - Time-dependent AUC: most relevant if we want to predict occurrence of an event in a period up to time t (rather than at a specific time t). Note that this only measures discrimination, not calibration.  
    - Time-dependent Brier score: can be used to assess both calibration and discrimination 


Todo:
- Models:
  - M1: null model that takes in X and returns 0.5 for every case
  - M2: perfect model that takes in X and passes it to the known Weibull survival
      function to get true survival probabilities (or 1-surv_prob if we want a risk
      score)
  - M3: intermediate model that passes to very similar Weibull params, but not exact
  - M4: KM model "learned" from training data
- set up functions for generating the data and splitting into train/test
- Evaluate models M1-M4 on test data


In [1]:
%%capture
!pip install scikit-survival